In [ ]:
#Import the acrtuals to compare to model guesses
#nc file
import netCDF4
import numpy as np
import time
import matplotlib.pyplot as plt
import xarray as xr
import dask
print('begin')

#Path for Aqua_Planet, 2 months, No VPB
path_to_file = "/fast/gmooers/Preprocessed_Data/7_Years_Spaced/full_physics_essentials_valid_month02_targets.nc"
ds = xr.open_dataset(path_to_file)
truths = ds.targets[:, 30:60].values
lons = ds.lon.values
lats = ds.lat.values
print('halfway')
#Path for Aqua_Planet, 2 months, No VPB
path_to_file = "/fast/gmooers/Models/Jordan_Best.nc"
ds = xr.open_dataset(path_to_file)
features = ds.Prediction[:, 30:60].values

x = 144
#x = 128
y = 96
#y=64
z = 30
t = int(len(truths)/(x*y))
print(t)

In [ ]:
reconstructed_targets = np.zeros(shape=(x, y, t, z))
reconstructed_features = np.zeros(shape=(x, y, t, z))
#print(reconstructed_targets.shape)
print('hi')
count = 0
for i in range(t):
    for j in range(y):
        for k in range(x):
            A = truths[count]
            B = features[count]
            reconstructed_targets[k, j, i, :] = A
            reconstructed_features[k, j, i, :] = B
            count = count + 1

print(reconstructed_targets.shape)
print(reconstructed_features.shape)

In [ ]:
predictions = np.nanmean(reconstructed_targets, axis=0)
truths = np.nanmean(reconstructed_features, axis=0)

daily_predictions = np.zeros(shape=(len(predictions), int(len(predictions[0])/96), len(predictions[0][0])))
daily_truths = np.zeros(shape=(len(truths), int(len(truths[0])/96), len(truths[0][0])))
daily_predictions[:,:,:] = np.nan
daily_truths[:,:,:] = np.nan

In [ ]:
current = 0

for i in range(int(len(truths[0])/96)):
    daily_predictions[:,i,:] = np.nansum(predictions[:,current:current+96,:], axis=1)
    daily_truths[:,i,:] = np.nansum(truths[:,current:current+96,:],axis=1)
    current = current+96

In [ ]:
R = np.zeros(shape=(y, z))
SSE = np.zeros(shape=(y, z))
SVAR = np.zeros(shape=(y, z))


for i in range(len(daily_predictions)):
    for j in range(len(daily_predictions[0][0])):
            target_array = np.squeeze(daily_truths[i,:,j])
            predict_array = np.squeeze(daily_predictions[i,:,j])
    
            sse = np.sum((target_array-predict_array)**2.0)
            svar = np.sum((target_array-np.mean(target_array))**2.0)
            r_2 = 1-(sse/svar)
            R[i, j] = r_2
            SSE[i, j] = sse
            SVAR[i, j] = svar
    
    if i%10==0:
        print(i)
        
R_Save = np.squeeze(R)
SSE_Save = np.squeeze(SSE)
SVAR_Save = np.squeeze(SVAR)

np.save("/fast/gmooers/Other_R2/Improved_Daily_SSE_SPCAM5_15_Min_Interval_Moistening.npy", SSE_Save)
np.save("/fast/gmooers/Other_R2/Improved_Daily_SVAR_SPCAM5_15_Min_Interval_Moistening.npy", SVAR_Save)
np.save("/fast/gmooers/Other_R2/Improved_Daily_SPCAM5_15_Min_Interval_Moistening.npy", R_Save)